In [ ]:
import sys
sys.path.insert(0, '/directory/tothe/handshakefile/')
sys.path.append('/home/ak/Documents/PaperCode/stylised_facts/')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import lob_for_futures as lobFut
from lob_for_futures import *
import os
import matplotlib.pyplot as plt

from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch
import pickle
import datetime as dt
from datetime import timedelta
import glob
import fathon
from fathon import fathonUtils as fu
import itertools
import pyinform as pyinf
# import pingouin as pig
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from scipy.special import kl_div
import time
import matplotlib.ticker as mtick
dataFolder = lobFut.dataFolder
quotes = [f for f in os.listdir(dataFolder) if str('_quotes') in f]
trades = [f for f in os.listdir(dataFolder) if str('_trades') in f]
elements = '/media/ak/Elements/OrderBookProcessedData'

procesSymbols = sorted(os.listdir(elements))
symbolIdx = 0

symbols = [f.split("_")[0] for f in quotes]
#
symbol = sorted(symbols)[symbolIdx]
print(symbol)
quotesFileCh = os.path.join(dataFolder, quotes[symbolIdx])
tradesFileCh = os.path.join(dataFolder, trades[symbolIdx])
#
# # get common Dates
quotesDates = sorted([f.split(".csv")[0] for f in os.listdir(quotesFileCh)])
tradesDates = sorted([f.split(".csv")[0] for f in os.listdir(tradesFileCh)])
intersectionDates = list(set(quotesDates).intersection(tradesDates))

trades_cols = ['size', 'time', 'type', 'value']
#experimentsDestination = '/media/ak/T7/MFDFA Experiments'
plt.style.use(os.path.join('/home/ak/.config/matplotlib','latexstyle.mplstyle'))

In [ ]:
dataFolder

In [ ]:
def returns(s):

    # compute log returns
    arr = np.diff(np.log(s))

    return (pd.Series(arr, index=s.index[1:]))

In [ ]:

symbols = sorted(os.listdir(elements))
procesSymbols

In [ ]:
# Lets create a function that shifted your dataframe first before calling the corr(). 
# Let us break down what we hope to accomplish, and then translate that into code. 
# For each hour (row) and for a given feature (column) I would like to find the value 
# for that feature N hours prior. 
# For each value of N (1-6 in our case) I want to make a new column 
# for that feature representing the Nth prior hour's measurement.

def df_derived_by_shift(df,lag=0,NON_DER=[]):
    df = df.copy()
    if not lag:
        return df
    cols ={}
    for i in range(1,lag+1):
        for x in list(df.columns):
            if x not in NON_DER:
                if not x in cols:
                    cols[x] = ['{}_{}'.format(x, i)]
                else:
                    cols[x].append('{}_{}'.format(x, i))
    for k,v in cols.items():
        columns = v
        dfn = pd.DataFrame(data=None, columns=columns, index=df.index)    
        i = 1
        for c in columns:
            dfn[c] = df[k].shift(periods=i)
            i+=1
        df = pd.concat([df, dfn], axis=1)
    return df

def get_test_stats(bar_types,bar_returns,test_func,*args,**kwds):
    
    dct = {bar:(int(bar_ret.shape[0]), test_func(bar_ret,*args,**kwds)) 
           for bar,bar_ret in zip(bar_types,bar_returns)}
    df = (pd.DataFrame.from_dict(dct)
          .rename(index={0:'sample_size',1:f'{test_func.__name__}_stat'})
          .T)
    return df

In [ ]:
symbolIdx = 1
procsdSymbolFolder = os.path.join(elements, procesSymbols[symbolIdx])
files = sorted(os.listdir(procsdSymbolFolder))
bars =['volume_bar', 'calendar_bar', 'usd_volume_bar', 'tick_bar']
symbol = symbols[symbolIdx]
print(symbol)

In [ ]:
filesIdx =0
fileLocation = os.path.join(procsdSymbolFolder, files[filesIdx])
openedDict = open_pickle_filepath(fileLocation)

In [ ]:
volume_bar_df = openedDict[list(openedDict.keys())[0]]

In [1]:
micro_df_volume =apply_micro_structure_features(volume_bar_df)

NameError: name 'apply_micro_structure_features' is not defined